In [74]:
# Install mysql module
!pip3.9 install mysql-connector-python
!pip3.9 install pandas

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [70]:
# Make sure ur IP is on file
!curl ipecho.net/plain


66.253.130.172

In [1]:
# Connect to server
import mysql.connector
import random
import pandas as pd

mydb = mysql.connector.connect(
    host="35.232.56.109",
    user="root",
    password="ah08EbAGxF04I503",
    database="SpotifyHelpr"
)

mycursor = mydb.cursor()

In [2]:
# Load CSV
df = pd.read_csv("SpotifyFeatures.csv")
ids = df.track_id.tolist()

# Get 1000 most popular songs
df2 = df.drop_duplicates(subset=["track_id"]).sort_values("popularity",ascending=False)

In [3]:
# Print results from query
def print_cursor(cursor):
    myresult = cursor.fetchall()
    for x in myresult:
        print(x)

In [4]:
# Commit changes to server
def commit():
    mydb.commit()

In [ ]:
# Create Users table
mycursor.execute("""CREATE TABLE Users (
	userId   INT         NOT NULL,
	userName VARCHAR(50),
	password VARCHAR(50),
	apiToken VARCHAR(50),

	PRIMARY KEY(userId)
);""")

In [ ]:
# Populate Users table
for i in range(1009):
    mycursor.execute(f'Insert into Users Values({i+1}, "test{i+1}", "pwd", "api{i+1}");')



In [ ]:
# Create the Songs table
mycursor.execute("""CREATE TABLE Songs (
	songId         VARCHAR(50) NOT NULL,
	name           VARCHAR(150),
	artist         VARCHAR(50),
	genre          VARCHAR(50),
	url            VARCHAR(50),
	likenessFactor REAL,

	PRIMARY KEY(songId)
);""")

In [13]:
# Populate Songs table
# idx = -1
completed = 22439
amt = 25000

to_add = df2.head(completed+amt).tail(amt)
l = []

for i, entry in to_add.iterrows():
    # idx += 1
    # if idx < 134:
    #     continue
    try:
        entry = list(entry)
        url = f"open.spotify.com/track/{entry[3]}"
        entry2 = entry[2].replace('"', '')
        mycursor.execute(f'Insert into Songs Values("{entry[3]}", "{entry2}", "{entry[1]}", "{entry[0]}", "{url}", {random.randint(1, 20)});')
        commit()
    except:
        continue

In [ ]:
# Create the Reviews table
mycursor.execute("""CREATE TABLE Reviews (
	reviewId INT          NOT NULL,
	rating   REAL,
	body     VARCHAR(140),

	songId   VARCHAR(50),
	userId   INT,

	PRIMARY KEY(reviewId),
	FOREIGN KEY(songId) REFERENCES Songs(songId) ON DELETE CASCADE,
	FOREIGN KEY(userId) REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Populate Reviews table
l=[]
d = {}
idx = 0
for i, entry in df2head.iterrows():
    d[idx] = entry[3]
    idx += 1

for i in range(1000):
    f = [d[random.randint(0, 999)], random.randint(0, 1000)]
    while(f in l):
        f = [d[random.randint(0, 999)], random.randint(0, 1000)]

    l.append(f)

    mycursor.execute(f'insert into Reviews values({i}, {random.randint(0, 10)}, "body{i}", "{f[0]}", {f[1]});')
    commit()

In [ ]:
# Create Playlists table
mycursor.execute("""CREATE TABLE Playlists (
	playlistId   INT         NOT NULL,
	playlistName VARCHAR(50),

	userId       INT,

	PRIMARY KEY(playlistId),
	FOREIGN KEY(userId) REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Populate Playlists table
for i in range(1000):
    mycursor.execute(f'Insert into Playlists Values({i}, "playlist{i}", {random.randint(0, 1000)});')
    commit()

In [ ]:
# Create SongsFoundIn table
mycursor.execute("""CREATE TABLE SongsFoundIn (
	songId     VARCHAR(50) NOT NULL,
	playlistId INT         NOT NULL,

	PRIMARY KEY(songId, playlistId),
	FOREIGN KEY(songId)     REFERENCES Songs(songId)         ON DELETE CASCADE,
	FOREIGN KEY(playlistId) REFERENCES Playlists(playlistId) ON DELETE CASCADE
);""")

In [ ]:
# Populate SongsFoundIn table
d = {}
idx = 0
for i, entry in df2head.iterrows():
    d[idx] = entry[3]
    idx += 1

for i in range(1000):
    l = []
    for j in range(random.randint(3, 8)):
        f = d[random.randint(0, 999)]
        while(f in l):
            f = d[random.randint(0, 999)]

        l.append(f)
        mycursor.execute(f'Insert into SongsFoundIn Values("{f}", {i});')
        commit()
    

In [ ]:
# Create table Friends
mycursor.execute("""CREATE TABLE Friends (
	friendId INT  NOT NULL,
	userId   INT  NOT NULL,
	fScore   REAL,

	PRIMARY KEY(friendId, userId),
	FOREIGN KEY(friendId) REFERENCES Users(userId) ON DELETE CASCADE,
	FOREIGN KEY(userId)   REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Create random friendships

friendships = []
for i in range(2000):
    f1 = random.randint(0, 999)
    f2 = random.randint(0, 999)
    while(f1 == f2):
        f2 = random.randint(0, 999)

    l = [f1, f2]
    l2 = [f2, f1]

    if l not in friendships and l2 not in friendships:
        mycursor.execute(f'Insert into Friends Values({l[1]}, {l[0]}, {random.randint(1, 10)});')
        friendships.append(l)
        

In [ ]:
# Find the name, artist, and average ratings for all the songs in a given playlist (eg. playlistId 700)
query = """SELECT name, artist, r.avg_rating
FROM Songs
NATURAL JOIN (SELECT songId FROM SongsFoundIn WHERE playlistId = 700) s
LEFT JOIN (SELECT songId, avg(rating) AS avg_rating FROM Reviews GROUP BY songId) r
ON Songs.songId = r.songId;"""

mycursor.execute(query) # Find the name, artist, and average ratings for all the songs in a given playlist (eg. playlistId 700)
print_cursor(mycursor)

In [ ]:
# Find the username, song name, artist, body, and ratings for all the reviews written by all of a User's friends (eg. userId = 90). We are using a UNION in order to find occurrences where the user is a friend of another user (as our friends relations are directed)
query = """SELECT Users.userName, sub.name, sub.artist, sub.body, sub.rating
FROM 
(SELECT Reviews.userId, Songs.name, Songs.artist, Reviews.body, Reviews.rating
	FROM Songs NATURAL JOIN Reviews) sub
JOIN Users ON Users.userId = sub.UserId
WHERE sub.userId IN (SELECT friendId FROM Friends WHERE userId = 90) 

UNION

SELECT Users.userName, sub.name, sub.artist, sub.body, sub.rating
FROM
(SELECT Reviews.userId, Songs.name, Songs.artist, Reviews.body, Reviews.rating
	FROM Songs NATURAL JOIN Reviews) sub 
JOIN Users ON Users.userId = sub.UserId
WHERE sub.userId IN (SELECT userId FROM Friends WHERE friendId = 90);"""

mycursor.execute(query) 
print_cursor(mycursor)

In [9]:
query = """SELECT * from Songs;"""

mycursor.execute(query) 
l=[]  # Stores all playlistIds of a User
print_cursor(mycursor)



('000xQL6tZNLJzIrtIgxqSl', 'Still Got Time', 'ZAYN', 'Dance', 'open.spotify.com/track/000xQL6tZNLJzIrtIgxqSl', 7.0)
('003eoIwxETJujVWmNFMoZy', 'Growing Pains', 'Alessia Cara', 'Dance', 'open.spotify.com/track/003eoIwxETJujVWmNFMoZy', 20.0)
('005lwxGU1tms6HGELIcUv9', 'I Kissed A Girl', 'Katy Perry', 'Dance', 'open.spotify.com/track/005lwxGU1tms6HGELIcUv9', 6.0)
('00B7TZ0Xawar6NZ00JFomN', 'Best Life (feat. Chance The Rapper)', 'Cardi B', 'Rap', 'open.spotify.com/track/00B7TZ0Xawar6NZ00JFomN', 17.0)
('00BNT97AtJ5aB8SSsE5xGH', 'Threat 2 Society', '2 Chainz', 'Hip-Hop', 'open.spotify.com/track/00BNT97AtJ5aB8SSsE5xGH', 2.0)
('00GOPLxW4PGQuUYdPJh8K1', 'IV. Sweatpants', 'Childish Gambino', 'Hip-Hop', 'open.spotify.com/track/00GOPLxW4PGQuUYdPJh8K1', 17.0)
('00i0O74dXdaKKdCrqHnfXm', 'La Mordidita', 'Ricky Martin', 'Dance', 'open.spotify.com/track/00i0O74dXdaKKdCrqHnfXm', 20.0)
('00KjOnN3U40e3lXFUOue7h', '10AM/Save The World (feat. Gucci Mane)', 'Metro Boomin', 'Hip-Hop', 'open.spotify.com/track/

In [14]:
query = """SELECT COUNT(*) from Songs;"""

mycursor.execute(query) 
l=[]  # Stores all playlistIds of a User
print_cursor(mycursor)



(22439,)


In [76]:
query = """SELECT playlistId FROM Playlists WHERE userId = 90;"""

mycursor.execute(query) 
l=[]  # Stores all playlistIds of a User
# print_cursor(mycursor)
myresult = mycursor.fetchall()
for x in myresult:
    l.append(x[0])


for i in l:
    
    print(i)
    query = f"""SELECT * FROM Songs WHERE songId IN (SELECT songId FROM SongsFoundIn WHERE playlistId = {i})"""

    mycursor.execute(query) 
    print_cursor(mycursor)

65
('00GOPLxW4PGQuUYdPJh8K1', 'IV. Sweatpants', 'Childish Gambino', 'Hip-Hop', 'open.spotify.com/track/00GOPLxW4PGQuUYdPJh8K1', 17.0)
('01OrNS0PUb0f1W9EacyxyR', 'Doctor Marihuana', 'Cartel De Santa', 'Hip-Hop', 'open.spotify.com/track/01OrNS0PUb0f1W9EacyxyR', 5.0)
('01sjbEk3X7KNXFvXXXN5MY', 'Baptiize', 'Future', 'Hip-Hop', 'open.spotify.com/track/01sjbEk3X7KNXFvXXXN5MY', 14.0)
('0IktbUcnAGrvD03AWnz3Q8', 'Lucky', 'Jason Mraz', 'Pop', 'open.spotify.com/track/0IktbUcnAGrvD03AWnz3Q8', 5.0)
('1uKjQoh8JZj9ryuYRhpd7E', 'Ahora Te Puedes Marchar', 'Luis Miguel', 'Pop', 'open.spotify.com/track/1uKjQoh8JZj9ryuYRhpd7E', 2.0)
('41zXlQxzTi6cGAjpOXyLYH', 'idontwannabeyouanymore', 'Billie Eilish', 'Pop', 'open.spotify.com/track/41zXlQxzTi6cGAjpOXyLYH', 5.0)
('4DTpngLjoHj5gFxEZFeD3J', 'Codeine Dreaming (feat. Lil Wayne)', 'Kodak Black', 'Hip-Hop', 'open.spotify.com/track/4DTpngLjoHj5gFxEZFeD3J', 2.0)
('5uvucCRBTkxv7VxIDLOqe7', 'Devuelveme', 'Ozuna', 'Reggaeton', 'open.spotify.com/track/5uvucCRBTkxv7VxI

In [ ]:
# Find the name, artist, and average ratings for all the songs in a given playlist (eg. playlistId 700)
query = """SELECT name, artist, r.avg_rating
FROM Songs
NATURAL JOIN (SELECT songId FROM SongsFoundIn WHERE playlistId = 0) s
LEFT JOIN (SELECT songId, avg(rating) AS avg_rating FROM Reviews GROUP BY songId) r
ON Songs.songId = r.songId;"""

#mycursor.execute("SHOW INDEX FROM Users")
mycursor.execute(query)
# mycursor.execute("ALTER TABLE Users ADD INDEX (userName);")  # Find the name, artist, and average ratings for all the songs in a given playlist (eg. playlistId 700)
print_cursor(mycursor)

In [ ]:
mycursor.execute("SHOW INDEX FROM Users")
print_cursor(mycursor)

In [ ]:
mycursor.execute("""EXPLAIN ANALYZE """ + query)
print_cursor(mycursor)